# Example 2: Metabolite workflow

This notebook provides insights on how to use the tool if you have list of metabolites.

# Import modules

In [ ]:
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.analyzer.summarize import BioGraph
from pyBiodatafuse.annotators import molmedb, opentargets, aopwiki
from pyBiodatafuse.graph import saver
from pyBiodatafuse.utils import combine_sources, create_or_append_to_metadata

# Load list of metabolites/chemicals/compounds

In [ ]:
metabolites_of_interest = """100208
10040286
10041551
10025195
5291
8571
697993
21831736
159603
445643
"""

metabolite_list = metabolites_of_interest.split("\n")
len(metabolite_list)

In [ ]:
data_input = pd.DataFrame(metabolite_list, columns=["identifier"])
data_input.head()

### Entity resolution using BridgeDB

In [ ]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="PubChem Compound",
    output_datasource="All",
)
bridgdb_df.head()

### Transporter inhibited from MolMeDB

In [ ]:
(
    molmedb_transporter_inhibited_df,
    molmedb_transporter_inhibited_metadata,
) = molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
molmedb_transporter_inhibited_df.head()

### Disease from OpenTargets

In [ ]:
(
    opentargets_df,
    opentargets_metadata,
) = opentargets.get_disease_compound_interactions(bridgedb_df=bridgdb_df)
opentargets_df.head()

### AOPs from AOP-Wiki

In [ ]:
(
    aopwiki_df,
    aopwiki_metadata,
) = aopwiki.get_aops(bridgedb_df=bridgdb_df)
aopwiki_df.head()

# Create the BDF graph

In [ ]:
combined_df = combine_sources(
    bridgdb_df, df_list=[opentargets_df, molmedb_transporter_inhibited_df]
)
combined_df.head()

In [ ]:
combined_metadata = create_or_append_to_metadata(
    bridgdb_metadata, [opentargets_metadata, molmedb_transporter_inhibited_metadata]
)

# Saving the graph

In [ ]:
import os

DATA_DIR = "./data/metabolite_workflow"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
pygraph = saver.save_graph(
    combined_df=combined_df,
    combined_metadata=combined_metadata,
    graph_name="metabolite_workflow",
    graph_dir=DATA_DIR,
)

# Graph statistics

In [ ]:
graph_obj = BioGraph(graph=pygraph)
graph_obj.graph_summary

In [ ]:
graph_obj.count_nodes_by_data_source(plot=True)

In [ ]:
graph_obj.count_edge_by_data_source(plot=True)